In [1]:
import psycopg2
import boto3
import pandas as pd
import yfinance as yf
import psycopg2.extras as extras

In [2]:
def connect_to_db() -> object:
    """
    Fetch DB username and password from AWS SSM and connect to RDS database.
    """
    aws_client = boto3.client("ssm")

    parameters = aws_client.get_parameters(
        Names=[
            "stock_twitter_db_user",
            "stock_twitter_db_password",
        ],
        WithDecryption=True,
    )

    credentials = {}
    for parameter in parameters["Parameters"]:
        credentials[parameter["Name"]] = parameter["Value"]

    db_host = "stock-twitter-db.cdx9enjjuwaj.us-east-1.rds.amazonaws.com"
    db_name = "postgres"
    db_port = "5432"
    db_user = credentials["stock_twitter_db_user"]
    db_pass = credentials["stock_twitter_db_password"]
    conn = psycopg2.connect(
        host=db_host, database=db_name, user=db_user, password=db_pass, port=db_port
    )

    return conn

In [3]:
conn = connect_to_db()

In [4]:
pd.read_sql("SELECT * FROM stock", con=conn)

c:\Users\Luca\Documents\Coding\Shopify-Project\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


DatabaseError: Execution failed on sql 'SELECT * FROM stock': relation "stock" does not exist
LINE 1: SELECT * FROM stock
                      ^


In [ ]:
def get_stock_data(ticker):
    raw_data = yf.download(tickers=ticker, period="1y", interval="1d")
    data = pd.DataFrame(columns=["ticker", "timestamp", "price"])
    data["timestamp"] = raw_data.index
    data["price"] = raw_data["Close"].values
    data['ticker'] = ticker
    return data

In [ ]:
df = get_stock_data("URA")
df

In [ ]:
def execute_values(conn, df, key, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s ON CONFLICT(%s) DO NOTHING" % (table, cols, key)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

In [ ]:
execute_values(conn, df, "ticker, timestamp", "stock_price")

In [7]:
pd.read_sql("SELECT * FROM stock_price", con=conn)

c:\Users\Luca\Documents\Coding\Shopify-Project\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ticker,timestamp,price
0,URA,2021-06-02,23.02
1,URA,2021-06-03,22.63
2,URA,2021-06-04,23.30
3,URA,2021-06-07,23.63
4,URA,2021-06-08,23.38
...,...,...,...
249,URA,2022-05-26,21.77
250,URA,2022-05-27,22.19
251,URA,2022-05-31,21.94
252,URA,2022-06-01,21.54


In [8]:
from utils.twitter_client import TwitterClient

In [9]:
def get_twitter_keys() -> dict:
    """Retrieve secrets from Parameter Store."""
    # Create our SSM Client.
    aws_client = boto3.client('ssm')

    # Get our keys from Parameter Store.
    parameters = aws_client.get_parameters(
        Names=[
            'twitter_api_key',
            'twitter_api_secret',
            'twitter_access_token',
            'twitter_access_secret'
        ],
        WithDecryption=True
    )

    # Convert list of parameters into simpler dict.
    keys = {}
    for parameter in parameters['Parameters']:
        keys[parameter['Name']] = parameter['Value']

    return keys

In [10]:
keys = get_twitter_keys()
api = TwitterClient(keys)
tweets = api.get_tweets(query = 'uranium', count = 1000)

In [11]:
tweets

[{'tweet_id': 1532448013653708800,
  'tweet_username': 'Jai Bharat 🇮🇳',
  'tweet_text': 'RT @ThePollLady: Did you know that Rafael Grossi, head of International Atomic Energy Agency let it slip at Davos Forum few days ago that U…',
  'tweet_created_at': '20220602-194412',
  'tweet_sentiment': 'negative'},
 {'tweet_id': 1532448010931609600,
  'tweet_username': 'Tanmay Dabre',
  'tweet_text': 'RT @ThePollLady: Did you know that Rafael Grossi, head of International Atomic Energy Agency let it slip at Davos Forum few days ago that U…',
  'tweet_created_at': '20220602-194411',
  'tweet_sentiment': 'negative'},
 {'tweet_id': 1532447976408367114,
  'tweet_username': 'UraniumCaesar ⚛',
  'tweet_text': 'RT @enCoreEnergy_EU: Saudi Arabia sent inquiries last week for construction of two 1.4 GW #nuclear power plants to South Korea, France, Chi…',
  'tweet_created_at': '20220602-194403',
  'tweet_sentiment': 'neutral'},
 {'tweet_id': 1532447737345630209,
  'tweet_username': 'Joshua Davidson',
  'tw